In [1]:
# Importamos la librerias
import requests
from bs4 import BeautifulSoup
import random
from datetime import datetime
import pandas as pd
import re

# Importamos Selenium
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import os

In [2]:
# Copiamos la url del periodico BIO BIO
URL=requests.get("https://www.biobiochile.cl/lista/busca-2020/categorias/nacional")
soup=BeautifulSoup(URL.text,'html.parser')

In [3]:
# Creamos una variable que contengo los resultados de busqueda con beautiful soup
Titulares_Nacionales = soup.find("div", {"class": "results-container"})
Titulares_Nacionales

<div class="results-container">
<div class="results">
<article class="article article-horizontal article-with-square justify-content-between">
<div class="article-content-container">
<div class="article-text-container">
<a href="https://www.biobiochile.cl/noticias/nacional/region-de-valparaiso/2022/09/20/repudio-provocan-carreras-de-perros-galgos-realizadas-en-zapallar-durante-fiestas-patrias.shtml">
<h2 class="article-title">Repudio provocan carreras de perros galgos realizadas en Zapallar durante Fiestas Patrias</h2>
</a>
<div class="article-author-container">
                                            Por
                                            <a class="article-author" href="https://www.biobiochile.cl/lista/busca-2020/autores/fdiaz">Felipe Díaz Montero</a>
<br/>La información es de
                                                                                                                                                        <a class="article-author" href="https://www.bi

In [4]:
# Buscamos lo que hay en cada artículo de noticia
Articulos = Titulares_Nacionales.find_all('article',{'class':'article article-horizontal article-with-square justify-content-between'})
Articulos

[<article class="article article-horizontal article-with-square justify-content-between">
 <div class="article-content-container">
 <div class="article-text-container">
 <a href="https://www.biobiochile.cl/noticias/nacional/region-de-valparaiso/2022/09/20/repudio-provocan-carreras-de-perros-galgos-realizadas-en-zapallar-durante-fiestas-patrias.shtml">
 <h2 class="article-title">Repudio provocan carreras de perros galgos realizadas en Zapallar durante Fiestas Patrias</h2>
 </a>
 <div class="article-author-container">
                                             Por
                                             <a class="article-author" href="https://www.biobiochile.cl/lista/busca-2020/autores/fdiaz">Felipe Díaz Montero</a>
 <br/>La información es de
                                                                                                                                                         <a class="article-author" href="https://www.biobiochile.cl/lista/busca-2020/autores/camil

In [5]:
# Creamos el driver para utilizar selenium y poder extraer el número de visitas
s = Service('C:\\Users\\colon\\Documents\\Magister Data Science\\Web Scrapping\\Proyecto Bio Bio\\chromedriver.exe')
driver = webdriver.Chrome(service=s)

In [6]:
# Creamos un ciclo for para extraer la variables que queremos. Utilizamos Beautiful Soup y Selenium
d = []
for articulo in range(len(Articulos)):
    try:
        headline = Articulos[articulo].find('h2',{'class':"article-title"}).get_text().replace('"',"")
        Autor = Articulos[articulo].find('a',{'class':'article-author'}).get_text()
        Fecha = Articulos[articulo].find('div',{'class':'article-date-hour'}).get_text().strip()
        Link  = Articulos[articulo].find('a', href = re.compile(r'[/]([a-z]|[A-Z])\w+'))['href']
        driver = webdriver.Chrome(service=s)
        driver.get(Link)
        visitas = driver.find_element(By.XPATH,"//span[@class='post-visits']").get_attribute('innerHTML')
        driver.close()

        d.append((headline,Autor,visitas,Fecha,Link))

        #print(headline,"//",Autor,'//',Fecha,"//",Link,"//",visitas)
        
    except:
       pass

In [7]:
# Convertimos el scraper en un Dataframe 
DF = pd.DataFrame(d,columns=('Headline','Autor','visitas','Fecha','Link'))

In [8]:
DF

,Headline,Autor,visitas,Fecha,Link
0,Repudio provocan carreras de perros galgos rea...,Felipe Díaz Montero,1262,"Martes 20 Septiembre, 2022 | 22:05",https://www.biobiochile.cl/noticias/nacional/r...
1,PDI investiga incendio de contenedores en fund...,Felipe Díaz Montero,424,"Martes 20 Septiembre, 2022 | 21:23",https://www.biobiochile.cl/noticias/nacional/r...
2,Minsal lanza test de antígeno a menos de $3 mi...,Diego Vera,776,"Martes 20 Septiembre, 2022 | 20:58",https://www.biobiochile.cl/noticias/nacional/c...
3,Carabineros desarticula banda criminal dedicad...,Felipe Díaz Montero,1055,"Martes 20 Septiembre, 2022 | 20:31",https://www.biobiochile.cl/noticias/nacional/r...
4,Hubo sentidas disculpas: Comunidad Judía de Ch...,Florencia Ortiz,1285,"Martes 20 Septiembre, 2022 | 20:14",https://www.biobiochile.cl/noticias/nacional/c...
5,Vandalizan dos iglesias católicas en la región...,Daniela Salgado,2062,"Martes 20 Septiembre, 2022 | 19:55",https://www.biobiochile.cl/noticias/nacional/r...
6,Gustito ideológico: diputados de oposición cue...,Jean Valencia,11.959,"Martes 20 Septiembre, 2022 | 19:51",https://www.biobiochile.cl/noticias/nacional/c...
7,Los Ríos: Autoridad Sanitaria decomisa más de ...,Manuel Cabrera,152,"Martes 20 Septiembre, 2022 | 19:35",https://www.biobiochile.cl/noticias/nacional/r...
8,Será inédito: Gobierno comenzará proceso de di...,Florencia Ortiz,577,"Martes 20 Septiembre, 2022 | 19:35",https://www.biobiochile.cl/noticias/nacional/c...
9,Urritiacoechea tilda de imprudente recurso pre...,Daniela Salgado,994,"Martes 20 Septiembre, 2022 | 19:20",https://www.biobiochile.cl/noticias/nacional/r...


In [9]:
# Creamos una copia para no repetir el scrapper
Df_copy = DF.copy()

In [11]:
# Hacemos un Split a la columna Fecha
Df_copy[['Fecha','Hora']] = Df_copy['Fecha'].str.split('|', expand = True)
Df_copy[['Fecha','Año']] = Df_copy['Fecha'].str.split(',',expand=True)
Df_copy[['Fecha','Día']] = Df_copy['Fecha'].str.split(' ',1,expand=True)
Df_copy[['Día','Mes']] = Df_copy['Día'].str.split(' ',1,expand=True)

In [12]:
# Creamoss la columna Date
Df_copy['Date'] = Df_copy["Fecha"] +' '+ Df_copy["Día"] +' ' +Df_copy['Mes']+ ',' +Df_copy['Año']

In [ ]:
#Convertimos las visitas en numeros enteros
Df_copy['visitas'] = Df_copy['visitas'].str.replace('.', '').astype('int64')

In [15]:
# Seleccionamos las columnas finales
Df_copy = Df_copy[['Headline','Autor','visitas','Date','Link']]

In [26]:
# Cambiamos el nombre a la columna Fecha
Df_copy.columns = ['Headline', 'Autor', 'Visitas', 'Fecha','Link']

In [28]:
#Visualizamos el Dataframe
Df_copy.head(3)

,Headline,Autor,Visitas,Fecha,Link
0,Repudio provocan carreras de perros galgos rea...,Felipe Díaz Montero,1262,"Martes 20 Septiembre, 2022",https://www.biobiochile.cl/noticias/nacional/r...
1,PDI investiga incendio de contenedores en fund...,Felipe Díaz Montero,424,"Martes 20 Septiembre, 2022",https://www.biobiochile.cl/noticias/nacional/r...
2,Minsal lanza test de antígeno a menos de $3 mi...,Diego Vera,776,"Martes 20 Septiembre, 2022",https://www.biobiochile.cl/noticias/nacional/c...


In [19]:
# Autores que generan más visitas 
round(Df_copy.groupby('Autor').agg('visitas').sum().sort_values(ascending=False))

Autor
Florencia Ortiz        25599
Jean Valencia          17475
Felipe Reyes            8024
María Jesús Vidal       5416
Daniela Salgado         3947
Felipe Díaz Montero     2741
Tamara Rojas            1521
Francisco Iturra         906
Diego Vera               776
Manuel Cabrera           376
Name: visitas, dtype: int64

In [20]:
# Numero de articulos publicado por autores
Df_copy['Autor'].value_counts()

Florencia Ortiz        4
Daniela Salgado        4
Felipe Díaz Montero    3
Jean Valencia          2
Manuel Cabrera         2
Felipe Reyes           1
Francisco Iturra       1
Tamara Rojas           1
María Jesús Vidal      1
Diego Vera             1
Name: Autor, dtype: int64

In [23]:
# 10 noticias más visitadas del día
Mas_vistas = Df_copy.head(10).sort_values(by='visitas',ascending=False)
print(Mas_vistas[['Headline','visitas']])

                                            Headline  visitas
6  Gustito ideológico: diputados de oposición cue...    11959
5  Vandalizan dos iglesias católicas en la región...     2062
4  Hubo sentidas disculpas: Comunidad Judía de Ch...     1285
0  Repudio provocan carreras de perros galgos rea...     1262
3  Carabineros desarticula banda criminal dedicad...     1055
9  Urritiacoechea tilda de imprudente recurso pre...      994
2  Minsal lanza test de antígeno a menos de $3 mi...      776
8  Será inédito: Gobierno comenzará proceso de di...      577
1  PDI investiga incendio de contenedores en fund...      424
7  Los Ríos: Autoridad Sanitaria decomisa más de ...      152


In [24]:
Df_copy.to_csv('AAAMartes_20.csv',encoding='utf-8-sig')